# **Importations**


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from gensim.models import Word2Vec

import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

# **Dataset Preparation**

In [2]:
#Load 1st Dataset of Amazon Reviews
Amazon_Dataframe_1 = pd.read_csv('../datasets/Amazon Reviews Dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv')
Amazon_Dataframe_1.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,https://www.newegg.com/Product/Product.aspx%25...
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,4,https://reviews.bestbuy.com/3545/5442403/revie...,Didnt know how much i'd use a kindle so went f...,Great for the price,johnnyjojojo,https://www.newegg.com/Product/Product.aspx%25...
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-10-09T00:00:00Z,True,177283626.0,3,5,https://redsky.target.com/groot-domain-api/v1/...,I am 100 happy with my purchase. I caught it o...,A Great Buy,Kdperry,https://www.newegg.com/Product/Product.aspx%25...
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/revie...,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack,https://www.newegg.com/Product/Product.aspx%25...


In [3]:
#Function to filter dataframe
def filter_dataframe(df, reviews, rating):
    df1 = df[[reviews, rating]]
    df1_filtered = df1[df1[rating] != 3]    
    sentiment_dict = {1:0, 2:0, 4:1, 5:1}    
    df1_filtered['sentiment'] = df1_filtered[rating].map(sentiment_dict)
    return df1_filtered

In [4]:
#Filtering Dataframe
Amazon_Dataframe_1 = filter_dataframe(Amazon_Dataframe_1 , "reviews.text", "reviews.rating")

C:\Users\Fakhar\AppData\Local\Temp/ipykernel_20396/2058486400.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_filtered['sentiment'] = df1_filtered[rating].map(sentiment_dict)


In [5]:
Amazon_Dataframe_1.head()

,reviews.text,reviews.rating,sentiment
1,This kindle is light and easy to use especiall...,5,1
2,Didnt know how much i'd use a kindle so went f...,4,1
3,I am 100 happy with my purchase. I caught it o...,5,1
4,Solid entry level Kindle. Great for kids. Gift...,5,1
5,This make an excellent ebook reader. Don't exp...,5,1


In [6]:
#Find the count of reviews in each sentiment category

Amazon_Dataframe_1.groupby(Amazon_Dataframe_1['sentiment']).count()

,reviews.text,reviews.rating
sentiment,,
0,117,117
1,4686,4686


In [7]:
#Load 2nd Dataset of Amazon Reviews

Amazon_Dataframe_2= pd.read_csv('../datasets/Amazon Reviews Dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

Amazon_Dataframe_2 = filter_dataframe(Amazon_Dataframe_2, "reviews.text", "reviews.rating")
Amazon_Dataframe_2.head()

C:\Users\Fakhar\AppData\Local\Temp/ipykernel_20396/2058486400.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_filtered['sentiment'] = df1_filtered[rating].map(sentiment_dict)


,reviews.text,reviews.rating,sentiment
1,Bulk is always the less expensive way to go fo...,4,1
2,Well they are not Duracell but for the price i...,5,1
3,Seem to work as well as name brand batteries a...,5,1
4,These batteries are very long lasting the pric...,5,1
5,Bought a lot of batteries for Christmas and th...,5,1


In [8]:
Amazon_Dataframe_2.groupby(Amazon_Dataframe_2['sentiment']).count()

,reviews.text,reviews.rating
sentiment,,
0,1581,1581
1,25545,25545


In [9]:
#Only negative reviews are required
Amazon_Dataframe_2 = Amazon_Dataframe_2[Amazon_Dataframe_2['sentiment'] == 0]
Amazon_Dataframe_2.count()

reviews.text      1581
reviews.rating    1581
sentiment         1581
dtype: int64

In [10]:
#Load 3rd Dataset of Amazon Reviews

Amazon_Dataframe_3 = pd.read_csv('../datasets/Amazon Reviews Dataset/1429_1.csv')

Amazon_Dataframe_3 = filter_dataframe(Amazon_Dataframe_3, "reviews.text", "reviews.rating")
Amazon_Dataframe_3.head()

C:\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Fakhar\AppData\Local\Temp/ipykernel_20396/2058486400.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_filtered['sentiment'] = df1_filtered[rating].map(sentiment_dict)


,reviews.text,reviews.rating,sentiment
0,This product so far has not disappointed. My c...,5.0,1.0
1,great for beginner or experienced person. Boug...,5.0,1.0
2,Inexpensive tablet for him to use and learn on...,5.0,1.0
3,I've had my Fire HD 8 two weeks now and I love...,4.0,1.0
4,I bought this for my grand daughter when she c...,5.0,1.0


In [11]:
Amazon_Dataframe_3.groupby(Amazon_Dataframe_3['sentiment']).count()

,reviews.text,reviews.rating
sentiment,,
0.0,812,812
1.0,32315,32316


In [12]:
#Get the negative reviews

Amazon_Dataframe_3 = Amazon_Dataframe_3[Amazon_Dataframe_3['sentiment'] == 0]

Amazon_Dataframe_3.count()

reviews.text      812
reviews.rating    812
sentiment         812
dtype: int64

In [13]:
#Concatenate all the dataframes of Amazon Reviews

Dataframe_list_1 = [ Amazon_Dataframe_1, Amazon_Dataframe_2, Amazon_Dataframe_3]

Amazon_Combined_Dataframe = pd.concat(Dataframe_list_1)

Amazon_Combined_Dataframe = Amazon_Combined_Dataframe.rename(columns={'reviews.text':'review','reviews.rating':'rating'})

In [14]:
Amazon_Combined_Dataframe

,review,rating,sentiment
1,This kindle is light and easy to use especiall...,5.0,1.0
2,Didnt know how much i'd use a kindle so went f...,4.0,1.0
3,I am 100 happy with my purchase. I caught it o...,5.0,1.0
4,Solid entry level Kindle. Great for kids. Gift...,5.0,1.0
5,This make an excellent ebook reader. Don't exp...,5.0,1.0
...,...,...,...
34654,This is exactly like any other usb power charg...,1.0,0.0
34656,Amazon should include this charger with the Ki...,1.0,0.0
34657,Love my Kindle Fire but I am really disappoint...,1.0,0.0
34658,I was surprised to find it did not come with a...,1.0,0.0


In [15]:
Amazon_Combined_Dataframe.groupby(Amazon_Combined_Dataframe['sentiment']).count()

,review,rating
sentiment,,
0.0,2510,2510
1.0,4686,4686


In [16]:
Amazon_Combined_Dataframe.count()

review       7196
rating       7196
sentiment    7196
dtype: int64

In [17]:
#Separating Negative Reviews From Amazon Reviews
Negative_Amazon_Dataframe = Amazon_Combined_Dataframe[Amazon_Combined_Dataframe['sentiment'] == 0]
Negative_Amazon_Dataframe

,review,rating,sentiment
20,I was looking for a kindle whitepaper. I saw o...,1.0,0.0
70,Looking at the picture and seeing it was 8th g...,1.0,0.0
80,it would not load my books proper. took a doze...,2.0,0.0
93,"The screen is too dark, and cannot adjust the ...",2.0,0.0
143,I have to say it was a little confusing and fr...,2.0,0.0
...,...,...,...
34654,This is exactly like any other usb power charg...,1.0,0.0
34656,Amazon should include this charger with the Ki...,1.0,0.0
34657,Love my Kindle Fire but I am really disappoint...,1.0,0.0
34658,I was surprised to find it did not come with a...,1.0,0.0


In [18]:
#Selecting Negative Reviews
Negative_Amazon_Dataframe = Negative_Amazon_Dataframe.iloc[0:2500]
Negative_Amazon_Dataframe.reset_index(drop=True, inplace=True)
Negative_Amazon_Dataframe 

,review,rating,sentiment
0,I was looking for a kindle whitepaper. I saw o...,1.0,0.0
1,Looking at the picture and seeing it was 8th g...,1.0,0.0
2,it would not load my books proper. took a doze...,2.0,0.0
3,"The screen is too dark, and cannot adjust the ...",2.0,0.0
4,I have to say it was a little confusing and fr...,2.0,0.0
...,...,...,...
2495,I previously owned the previous model. There i...,2.0,0.0
2496,Didn't work well in 4K at all. After going in ...,1.0,0.0
2497,Does not load up CNN or Travel channel.I even ...,2.0,0.0
2498,We had two AppleTVs - one in the master bedroo...,2.0,0.0


In [19]:
#Separating Positive Reviews
Positive_Amazon_Dataframe = Amazon_Combined_Dataframe[Amazon_Combined_Dataframe['sentiment'] == 1]
Positive_Amazon_Dataframe

,review,rating,sentiment
1,This kindle is light and easy to use especiall...,5.0,1.0
2,Didnt know how much i'd use a kindle so went f...,4.0,1.0
3,I am 100 happy with my purchase. I caught it o...,5.0,1.0
4,Solid entry level Kindle. Great for kids. Gift...,5.0,1.0
5,This make an excellent ebook reader. Don't exp...,5.0,1.0
...,...,...,...
4995,This is a great tablet for the price. Amazon i...,5.0,1.0
4996,This tablet is the perfect size and so easy to...,5.0,1.0
4997,Purchased this for my son. Has room to upgrade...,4.0,1.0
4998,I had some thoughts about getting this for a 5...,5.0,1.0


In [20]:
#Selecting Positive Reviews
Positive_Amazon_Dataframe = Positive_Amazon_Dataframe.iloc[0:2500]
Positive_Amazon_Dataframe.reset_index(drop=True, inplace=True)
Positive_Amazon_Dataframe 

,review,rating,sentiment
0,This kindle is light and easy to use especiall...,5.0,1.0
1,Didnt know how much i'd use a kindle so went f...,4.0,1.0
2,I am 100 happy with my purchase. I caught it o...,5.0,1.0
3,Solid entry level Kindle. Great for kids. Gift...,5.0,1.0
4,This make an excellent ebook reader. Don't exp...,5.0,1.0
...,...,...,...
2495,Absolutely love Kindle Kids Edition. Easy to u...,5.0,1.0
2496,"The grandkids love this tablet, easy for them ...",4.0,1.0
2497,Easy to use. I would recommend it to anyone wh...,5.0,1.0
2498,Bought for my son for Easter. He loves it and ...,5.0,1.0


In [21]:
#Final Amazon Reviews Dataframe
Dataframe_list_2 = [ Positive_Amazon_Dataframe, Negative_Amazon_Dataframe]
Final_Amazon_Dataframe = pd.concat(Dataframe_list_2)
Final_Amazon_Dataframe

,review,rating,sentiment
0,This kindle is light and easy to use especiall...,5.0,1.0
1,Didnt know how much i'd use a kindle so went f...,4.0,1.0
2,I am 100 happy with my purchase. I caught it o...,5.0,1.0
3,Solid entry level Kindle. Great for kids. Gift...,5.0,1.0
4,This make an excellent ebook reader. Don't exp...,5.0,1.0
...,...,...,...
2495,I previously owned the previous model. There i...,2.0,0.0
2496,Didn't work well in 4K at all. After going in ...,1.0,0.0
2497,Does not load up CNN or Travel channel.I even ...,2.0,0.0
2498,We had two AppleTVs - one in the master bedroo...,2.0,0.0


In [22]:
Final_Amazon_Dataframe.groupby(Final_Amazon_Dataframe['sentiment']).count()

,review,rating
sentiment,,
0.0,2500,2500
1.0,2500,2500


In [23]:
#Loading Product Reviews from json file

Product_Reviews_Dataframe = pd.read_json("../datasets/product_reviews.json",  lines=True)
Product_Reviews_Dataframe

,reviewText,reviewSummary,reviewRating
0,Gave this to my dad for a gag gift after direc...,more like funchuck,3
1,I hope a lot of people hear this cd. We need m...,Inspiring,5
2,I'm reading a lot of reviews saying that this ...,The best soundtrack ever to anything.,5
3,The music of Yasunori Misuda is without questi...,Chrono Cross OST,4
4,Probably the greatest soundtrack in history! U...,Too good to be true,5
...,...,...,...
999995,I bought this as a practice crossbow and the s...,Cheap,1
999996,THis cap does not fit to any of my Mamiya lens...,Does not fit for all RB67 lenses,1
999997,"When you elimate any positive attributes, all ...",The worst holmes movie,1
999998,This is a knock-off of the recent Hollywood ve...,Huh?,1


In [24]:
#FIltering Product Reviews
Filtered_Product_Reviews_Dataframe = filter_dataframe(Product_Reviews_Dataframe, "reviewText", "reviewRating")

C:\Users\Fakhar\AppData\Local\Temp/ipykernel_20396/2058486400.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_filtered['sentiment'] = df1_filtered[rating].map(sentiment_dict)


In [25]:
#Find the count of reviews in each sentiment category

Filtered_Product_Reviews_Dataframe .groupby(Filtered_Product_Reviews_Dataframe ['sentiment']).count()

,reviewText,reviewRating
sentiment,,
0,400000,400000
1,400000,400000


In [26]:
#Separating Positive Reviews
Positive_Product_Reviews_Dataframe = Filtered_Product_Reviews_Dataframe[Filtered_Product_Reviews_Dataframe['sentiment'] == 1]
Positive_Product_Reviews_Dataframe 

,reviewText,reviewRating,sentiment
1,I hope a lot of people hear this cd. We need m...,5,1
2,I'm reading a lot of reviews saying that this ...,5,1
3,The music of Yasunori Misuda is without questi...,4,1
4,Probably the greatest soundtrack in history! U...,5,1
5,"There's a reason this CD is so expensive, even...",5,1
...,...,...,...
993734,A very nice shoe & great looking. I had to ret...,4,1
993737,"I very seldom read a book I don't care for, an...",4,1
993739,I'll admit it - I've slowly quit reading stock...,4,1
993742,"An intriguing book, but not up there with the ...",4,1


In [27]:
#Separating Negative Reviews
Negative_Product_Reviews_Dataframe = Filtered_Product_Reviews_Dataframe[Filtered_Product_Reviews_Dataframe['sentiment'] == 0]
Negative_Product_Reviews_Dataframe 

,reviewText,reviewRating,sentiment
6,"This is a self-published book, and if you want...",1,0
8,A complete waste of time. Typographical errors...,1,0
9,I guess you have to be a romance novel lover f...,1,0
10,I feel I have to write to keep others from was...,1,0
14,sizes are much smaller than what is recomended...,2,0
...,...,...,...
999995,I bought this as a practice crossbow and the s...,1,0
999996,THis cap does not fit to any of my Mamiya lens...,1,0
999997,"When you elimate any positive attributes, all ...",1,0
999998,This is a knock-off of the recent Hollywood ve...,1,0


In [28]:
#Selecting Positive Reviews
Final_Positive_Product_Reviews_Dataframe = Positive_Product_Reviews_Dataframe.iloc[0:347500]
Final_Positive_Product_Reviews_Dataframe 

,reviewText,reviewRating,sentiment
1,I hope a lot of people hear this cd. We need m...,5,1
2,I'm reading a lot of reviews saying that this ...,5,1
3,The music of Yasunori Misuda is without questi...,4,1
4,Probably the greatest soundtrack in history! U...,5,1
5,"There's a reason this CD is so expensive, even...",5,1
...,...,...,...
861963,The only reason why I purchased this tape was ...,5,1
861968,This CD is really great! It can lift up any ch...,5,1
861970,who is Adam?this book will give you some clues...,4,1
861971,"I've a long ""to do"" list. A lot of my energy a...",5,1


In [29]:
#Selecting Negative Reviews
Final_Negative_Product_Reviews_Dataframe = Negative_Product_Reviews_Dataframe.iloc[0:347500]
Final_Negative_Product_Reviews_Dataframe 

,reviewText,reviewRating,sentiment
6,"This is a self-published book, and if you want...",1,0
8,A complete waste of time. Typographical errors...,1,0
9,I guess you have to be a romance novel lover f...,1,0
10,I feel I have to write to keep others from was...,1,0
14,sizes are much smaller than what is recomended...,2,0
...,...,...,...
879988,My wife ordered this book because of all the h...,2,0
879989,This was possibly the most tedious and depress...,1,0
879990,I suppose that I have to give the author credi...,1,0
879991,I hated this book. It was completely manic and...,1,0


In [30]:
#Concatenate Positive and Negative dataframes

Dataframe_list_3 = [Final_Positive_Product_Reviews_Dataframe , Final_Negative_Product_Reviews_Dataframe  ]

Final_Product_Reviews_Dataframe = pd.concat(Dataframe_list_3)

Final_Product_Reviews_Dataframe = Final_Product_Reviews_Dataframe.rename(columns={'reviewText':'review','reviewRating':'rating'})

Final_Product_Reviews_Dataframe

,review,rating,sentiment
1,I hope a lot of people hear this cd. We need m...,5,1
2,I'm reading a lot of reviews saying that this ...,5,1
3,The music of Yasunori Misuda is without questi...,4,1
4,Probably the greatest soundtrack in history! U...,5,1
5,"There's a reason this CD is so expensive, even...",5,1
...,...,...,...
879988,My wife ordered this book because of all the h...,2,0
879989,This was possibly the most tedious and depress...,1,0
879990,I suppose that I have to give the author credi...,1,0
879991,I hated this book. It was completely manic and...,1,0


In [31]:
#Find the count of reviews in each sentiment category

Final_Product_Reviews_Dataframe.groupby(Final_Product_Reviews_Dataframe['sentiment']).count()

,review,rating
sentiment,,
0,347500,347500
1,347500,347500


In [32]:
#Final Dataframe/Dataset

Dataframe_list_4 = [Final_Amazon_Dataframe, Final_Product_Reviews_Dataframe]

Final_Dataframe = pd.concat(Dataframe_list_4)
Final_Dataframe

,review,rating,sentiment
0,This kindle is light and easy to use especiall...,5.0,1.0
1,Didnt know how much i'd use a kindle so went f...,4.0,1.0
2,I am 100 happy with my purchase. I caught it o...,5.0,1.0
3,Solid entry level Kindle. Great for kids. Gift...,5.0,1.0
4,This make an excellent ebook reader. Don't exp...,5.0,1.0
...,...,...,...
879988,My wife ordered this book because of all the h...,2.0,0.0
879989,This was possibly the most tedious and depress...,1.0,0.0
879990,I suppose that I have to give the author credi...,1.0,0.0
879991,I hated this book. It was completely manic and...,1.0,0.0


In [33]:
#Final Data Count
Final_Dataframe.groupby(Final_Dataframe['sentiment']).count()

,review,rating
sentiment,,
0.0,350000,350000
1.0,350000,350000


In [34]:
# Split data into training set and testing set
X_train, X_test, Y_train, Y_test = train_test_split(Final_Dataframe['review'], Final_Dataframe['sentiment'], test_size=0.3, random_state=0)

print('Load', X_train.shape[0], 'training examples and', X_test.shape[0], 'testing examples. \n')
print('Show a review in the training set : \n', X_train.iloc[100])

Load 490000 training examples and 210000 testing examples. 

Show a review in the training set : 
 Really happy with everything...but not even one year and the lamp burned out and tv won't turn on. It's on about less than 3 hours a day!Very disappointing, thank God I live in Los Angeles where there are services techs available...just hope it all works out!


# **Text Preprocessing**

In [35]:
#Method for cleaning data
def cleanText(raw_text, split_text=False):
    letters_only = " "
    for char in str(raw_text):
      if char.isalpha() or char==" ": #Only letters
        letters_only += char
    words = letters_only.lower().split() # convert to lower case 
    if split_text==True:  # split text
        return (words)
    return( " ".join(words))

In [36]:
# Preprocessing Training Data & Testing Data
X_train_cleaned = []
X_test_cleaned = []

for review in X_train:
  X_train_cleaned.append(cleanText(review))
print('Show a cleaned review in the training set : \n',  X_train_cleaned[100])
    
for review in X_test:
    X_test_cleaned.append(cleanText(review))

Show a cleaned review in the training set : 
 really happy with everythingbut not even one year and the lamp burned out and tv wont turn on its on about less than hours a dayvery disappointing thank god i live in los angeles where there are services techs availablejust hope it all works out


In [37]:
# Splitting review text into parsed sentences uisng NLTK's punkt tokenizer
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def parseToSentences(review, tokenizer):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(cleanText(raw_sentence, split_text=True))
    return sentences


# Parse each review in the training set into sentences
sentences = []
for review in X_train_cleaned:
    sentences += parseToSentences(review, tokenizer)
    
print(len(sentences) ,'parsed sentences are in the training set\n')
print('Show a parsed sentence in the training set : \n',  sentences[10])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Fakhar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


489997 parsed sentences are in the training set

Show a parsed sentence in the training set : 
 ['i', 'cant', 'wait', 'for', 'their', 'next', 'cd', 'these', 'fantastic', 'young', 'handsome', 'heartthrobbing', 'male', 'tenors', 'are', 'absolutely', 'what', 'the', 'classical', 'world', 'needs', 'to', 'get', 'interest', 'in', 'classical', 'music', 'excited', 'and', 'panting', 'for', 'more', 'i', 'heard', 'this', 'cd', 'at', 'my', 'friends', 'house', 'who', 'purchased', 'it', 'on', 'her', 'vacation', 'to', 'england', 'i', 'didnt', 'want', 'to', 'go', 'homei', 'couldnt', 'get', 'enough', 'the', 'music', 'the', 'voices', 'brought', 'me', 'to', 'my', 'knees', 'im', 'a', 'big', 'fan', 'of', 'josh', 'grobin', 'but', 'these', 'guys', 'are', 'like', 'having', 'a', 'variety', 'of', 'deserts', 'and', 'being', 'unable', 'to', 'pick', 'one', 'you', 'want', 'them', 'all', 'i', 'defnitely', 'want', 'more', 'the', 'three', 'tenors', 'although', 'older', 'and', 'more', 'experienced', 'leave', 'you', 'unf

# **Word To Vector Model Creation**

In [38]:
#Creating word to vector model of parsed sentences of the reviews

word2vec_model = Word2Vec(
    vector_size=300,
    window=10,
    min_count=10,
    workers=4,
)

word2vec_model.build_vocab(sentences)

print("Training Word2Vec model ...\n")
word2vec_model.train(sentences, total_examples=word2vec_model.corpus_count, epochs=10)

Training Word2Vec model ...



(265870542, 367165660)

In [39]:
#Save trained word2vec model

word2vec_model.save("../models/Word2Vec.model") 

In [40]:
# Load trained Word2Vec model
Word2Vec_Train_Model = Word2Vec.load("../models/Word2Vec.model")


# Get Word2Vec embedding matrix
embedding_matrix = Word2Vec_Train_Model.wv.vectors  # embedding matrix, type = numpy.ndarray 
print("Shape of embedding matrix : ", embedding_matrix.shape) # (vocabulary size, embedding dimension)

Shape of embedding matrix :  (49148, 300)


# **Vectorizing X_train and X_test**

In [41]:
# Vectorize X_train and X_test
tokenizer = Tokenizer(num_words=embedding_matrix.shape[0]) #Create internal vocabulary
tokenizer.fit_on_texts(X_train_cleaned) #IMPORTANT #Creates word index dictionary e.g, word_index["the"]=1

# saving tokenizer
with open('../models/tokenizer.pickle', 'wb') as fileObject:
    pickle.dump(tokenizer, fileObject, protocol=pickle.HIGHEST_PROTOCOL)

#Creating Sequences
sequences_train = tokenizer.texts_to_sequences(X_train_cleaned)
sequences_test = tokenizer.texts_to_sequences(X_test_cleaned)

#Padding Sequences
X_train_seq = sequence.pad_sequences(sequences_train, maxlen=200)
X_test_seq = sequence.pad_sequences(sequences_test, maxlen=200)


# one-hot encoding of y_train and y_test
Y_train_seq = np_utils.to_categorical(Y_train, 2)
Y_test_seq = np_utils.to_categorical(Y_test, 2)

print('X_train shape:', X_train_seq.shape) 
print('X_test shape:', X_test_seq.shape) 
print('Y_train shape:', Y_train_seq.shape) 
print('Y_test shape:', Y_test_seq.shape) 

X_train shape: (490000, 200)
X_test shape: (210000, 200)
Y_train shape: (490000, 2)
Y_test shape: (210000, 2)


# **Implementation of LSTM Model**

In [42]:
#Implementing Deep Learning Model

#Preparing embedding layer
embedding_layer = Embedding( input_dim = embedding_matrix.shape[0], 
                            output_dim = embedding_matrix.shape[1], 
                            weights = [embedding_matrix],
                            input_length = 200 )


# Adding Embedding Layer, LSTM Layers, Dropout Layer & Dense Layers to the Model
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(128, return_sequences=True)) 
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3)) 
model.add(Dense(2, activation='softmax'))
model.summary()

# Compiling model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#Fitting data to the model
model.fit(X_train_seq, Y_train_seq, batch_size=32, epochs=3 , verbose=1)


# Model evaluation
score = model.evaluate(X_test_seq, Y_test_seq, batch_size=32)
print('Test loss : {:.4f}'.format(score[0]))
print('Test accuracy : {:.4f}'.format(score[1]))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 300)          14744400  
_________________________________________________________________
lstm (LSTM)                  (None, 200, 128)          219648    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 15,112,402
Trainable params: 15,112,402
Non-trainable params: 0
____________________________________________

In [43]:
#Saving Review Analysis Model
model.save("../models/review_analysis_model.h5")